In [1]:
import pandas as pd

from ltlcross_wrapper import Modulizer, ResAnalyzer
from tools_seminator_variants import tools
from tools import benchmark_names as names

In [2]:
res_dir = "data.seminator_variants"
!mkdir -p {res_dir}

In [3]:
def gather_cumulative(benchmarks, tools=None, transpose=True, **kwargs):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = pd.DataFrame(b.cumulative(tool_set=tools, highlight=False, **kwargs))
        tmp.columns = [name]
        data = data.append(tmp.transpose())
    if transpose:
        return data.transpose().style.apply(highlight_min, axis=0)
    else:
        return data.style.apply(highlight_min, axis=1)
    
def gather_mins(benchmarks, tools=None, transpose=True):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = b.min_counts(tool_set=tools)
        tmp.columns = pd.MultiIndex.from_tuples([(name, c) for c in tmp.columns])
        data = data.append(tmp.transpose(),sort=False).fillna(0)
    if transpose:
        return data.transpose().style.apply(highlight_max, axis=0)
    return data.style.apply(highlight_max, axis=1)

def highlight_min(s):    
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]

def highlight_max(s):    
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

# Run ltlcross

In [4]:
for name in names:
    print(name)
    m = Modulizer(tools, formula_file=f"../formulae/{name}.ltl",
                  name=f"SV.{name}", tmp_dir=f"{res_dir}/{name}.parts")
    m.split_task()
    m.resume()

literature_det
Parts to finish:
	[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151]
literature_sd
Parts to finish:
	[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
literature_nd
Parts to finish:
	[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18

In [5]:
!mv SV.* {res_dir}

## What is the most useful pipeline?

In [6]:
tool_set = [t for t in tools.keys() if t.startswith("spot") or t.startswith("pldi")]
benchmarks = {}
for name in names:
    b = ResAnalyzer(f"{res_dir}/SV.{name}.csv", tool_set=tool_set, cols=["states","time","acc","transitions","edges"])
    b.name = name
    benchmarks[name] = b

In [7]:
gather_cumulative(benchmarks, tools=["best"] + tool_set)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
best,705,307,308,2475,2828,5375
pldi#sba,711,325,352,2491,3182,6581
pldi#tba,705,320,327,2475,3111,6063
pldi#tgba,705,320,362,2475,3111,6220
spot#sba,711,318,425,2491,2931,6305
spot#tba,705,308,387,2475,2856,5877
spot#tgba,705,308,410,2475,2856,6250


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [8]:
gather_mins(benchmarks)

In [21]:
b = benchmarks["random_nd"]

In [22]:
b.cross_compare()

,spot#tgba,pldi#tgba,spot#tba,pldi#tba,spot#sba,pldi#sba,V
spot#tgba,nan,146,17,145,160,251,719
pldi#tgba,92,nan,64,19,168,161,504
spot#tba,63,186,nan,156,175,267,847
pldi#tba,108,57,71,nan,180,173,589
spot#sba,93,178,72,163,nan,172,678
pldi#sba,107,83,86,63,53,nan,392


In [10]:
b.bokeh_scatter_plot("spot#tba", "spot#tgba")

Loading BokehJS ...

Figure(id='1002', ...)

In [11]:
b.bokeh_scatter_plot("pldi#tba", "pldi#tgba")

Loading BokehJS ...

Figure(id='1124', ...)

In [12]:
b.bokeh_scatter_plot("pldi#tba","spot#tba", include_equal=True)

Loading BokehJS ...

Figure(id='1256', ...)

## Can we save using `cut-on-SCC-enytry`? Can we save using `powerset-on-cut=0`?

In [13]:
tool_set = [t for t in tools.keys() if t.find("power") >=0]
print(tool_set)

['entry.nopower', 'entry.power', 'always.nopower', 'always.power']


In [14]:
opt_benchmarks = {}
for name in names:
    b = ResAnalyzer(f"{res_dir}/SV.{name}.csv", tool_set=tool_set, cols=["states","time","acc","transitions","edges"])
    b.name = name
    opt_benchmarks[name] = b

In [15]:
gather_cumulative(opt_benchmarks, tools=tool_set)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
always.nopower,705,307,368,2475,2828,5543
always.power,705,307,308,2475,2828,5288
entry.nopower,705,307,371,2475,2828,5921
entry.power,705,307,318,2475,2828,5510


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [16]:
gather_mins(opt_benchmarks)

In [17]:
b = opt_benchmarks["random_nd"]
b.bokeh_scatter_plot("always.power","entry.power")

Loading BokehJS ...

Figure(id='1398', ...)

In [18]:
b.bokeh_scatter_plot("always.power","always.nopower")

Loading BokehJS ...

Figure(id='1550', ...)

In [19]:
b.cross_compare()

,entry.nopower,entry.power,always.nopower,always.power,V
entry.nopower,nan,29,46,63,138
entry.power,76,nan,86,45,207
always.nopower,121,124,nan,49,294
always.power,151,103,61,nan,315
